# チャットモデルで関数を呼び出す方法

このノートブックでは、GPTモデルの機能を拡張するために、Chat Completions APIと外部関数を組み合わせて使用する方法について説明します。

`tools`は、Chat Completion APIのオプションパラメータで、関数の仕様を提供するために使用できます。これの目的は、提供された仕様に準拠した関数引数をモデルが生成できるようにすることです。なお、APIは実際には関数呼び出しを実行しません。モデルの出力を使用して関数呼び出しを実行するのは開発者の責任です。

`tools`パラメータ内で、`functions`パラメータが提供された場合、デフォルトでモデルは関数の1つを使用することが適切なタイミングを決定します。`tool_choice`パラメータを`{"type": "function", "function": {"name": "my_function"}}`に設定することで、APIに特定の関数を強制的に使用させることができます。また、`tool_choice`パラメータを`"none"`に設定することで、APIに関数を一切使用しないよう強制することもできます。関数が使用された場合、出力にはレスポンス内に`"finish_reason": "tool_calls"`が含まれ、さらに関数名と生成された関数引数を持つ`tool_calls`オブジェクトが含まれます。

### 概要

このノートブックには以下の2つのセクションが含まれています：

- **関数引数を生成する方法：** 関数のセットを指定し、APIを使用して関数引数を生成する。
- **モデルが生成した引数で関数を呼び出す方法：** モデルが生成した引数で実際に関数を実行してループを閉じる。

## 関数引数の生成方法

In [1]:
!pip install scipy --quiet
!pip install tenacity --quiet
!pip install tiktoken --quiet
!pip install termcolor --quiet
!pip install openai --quiet

In [2]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

GPT_MODEL = "gpt-5"
client = OpenAI()

### ユーティリティ

まず、Chat Completions APIへの呼び出しを行い、会話の状態を維持・追跡するためのいくつかのユーティリティを定義しましょう。

In [3]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [4]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("tool_calls"):
            print(colored(f"assistant: {message['tool_calls']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("tool_calls"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### 基本概念

仮想的な天気APIとインターフェースするための関数仕様をいくつか作成しましょう。これらの関数仕様をChat Completions APIに渡して、仕様に準拠した関数引数を生成します。

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this unit from the forecast location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this unit from the forecast location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

モデルに現在の天気について質問すると、いくつかの確認質問で応答します。

In [6]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
messages.append(chat_response.choices[0].message.to_dict())
pretty_print_conversation(messages)


system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Sure—what city and state (or country) should I check? Also, do you prefer Celsius or Fahrenheit?



不足している情報を提供すると、適切な関数引数を生成してくれます。

In [7]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, tools=tools
)
messages.append(chat_response.choices[0].message.to_dict())
pretty_print_conversation(messages)


system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Sure—what city and state (or country) should I check? Also, do you prefer Celsius or Fahrenheit?

user: I'm in Glasgow, Scotland.

assistant: [{'id': 'call_k2QgGc9GT9WjxD76GvR0Ot8q', 'function': {'arguments': '{"location": "Glasgow, Scotland", "format": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_RtnXV5t49lqbWwhvGoEPZ7KY', 'function': {'arguments': '{"location": "Glasgow, Scotland", "format": "celsius", "num_days": 1}', 'name': 'get_n_day_weather_forecast'}, 'type': 'function'}]



異なるプロンプトを与えることで、私たちが教えた他の関数をターゲットにするように指示できます。

In [8]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
messages.append(chat_response.choices[0].message.to_dict())
pretty_print_conversation(messages)


system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Glasgow, Scotland over the next x days

assistant: How many days would you like the forecast for in Glasgow, Scotland? For example: 3, 5, 7, 10, or 14.



再び、モデルは十分な情報がまだないため、私たちに明確化を求めています。この場合、モデルは既に予報の場所を知っていますが、予報に何日分が必要かを知る必要があります。

In [9]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
messages.append(chat_response.choices[0].message.to_dict())
pretty_print_conversation(messages)


system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Glasgow, Scotland over the next x days

assistant: How many days would you like the forecast for in Glasgow, Scotland? For example: 3, 5, 7, 10, or 14.

user: 5 days

assistant: [{'id': 'call_lNzOVLrNSaSVjL3O3bN110af', 'function': {'arguments': '{"location":"Glasgow, Scotland","format":"celsius","num_days":5}', 'name': 'get_n_day_weather_forecast'}, 'type': 'function'}]



#### 特定の関数の使用を強制する、または関数を使用しない

モデルに特定の関数を使用するよう強制することができます。例えば、`function_call`引数を使用して`get_n_day_weather_forecast`を使用するよう強制できます。そうすることで、モデルにその関数の使用方法について仮定を立てるよう強制します。

In [10]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
messages.append(chat_response.choices[0].message.to_dict())
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me a weather report for Toronto, Canada.

assistant: [{'id': 'call_3hoMjl55OQ7LxfwhFyjxwv1T', 'function': {'arguments': '{"location":"Toronto, Canada","format":"celsius","num_days":5}', 'name': 'get_n_day_weather_forecast'}, 'type': 'function'}]



In [11]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
messages.append(chat_response.choices[0].message.to_dict())
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me a weather report for Toronto, Canada.

assistant: [{'id': 'call_wv5mdjEQJnBPuSci3xw09Tom', 'function': {'arguments': '{"location":"Toronto, ON","format":"celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}]



モデルに関数を一切使用しないよう強制することもできます。そうすることで、適切な関数呼び出しを生成することを防ぐことができます。

In [12]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
messages.append(chat_response.choices[0].message.to_dict())
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me the current weather (use Celcius) for Toronto, Canada.

assistant: I don’t have live access to pull the current conditions right now. To get Toronto’s current weather in Celsius, check any of these quickly:
- Environment Canada: weather.gc.ca (search “Toronto”)
- The Weather Network: theweathernetwork.com/ca/weather/ontario/toronto
- Google: search “Toronto weather” (shows °C by default in Canada)
- AccuWeather or Weather.com (set units to °C)

If you paste the current readings here (temperature, feels-like, wind, precipitation), I can interpret them and advise on what to wear or plan for.



### 並列関数呼び出し

gpt-5、gpt-4.1、gpt-4oなどの新しいモデルは、1回のターンで複数の関数を呼び出すことができます。

In [13]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model="gpt-4o"
)

assistant_message = chat_response.choices[0].message.tool_calls
assistant_message

[ChatCompletionMessageFunctionToolCall(id='call_KlZ3Fqt3SviC6o66dVMYSa2Q', function=Function(arguments='{"location": "San Francisco, CA", "format": "fahrenheit", "num_days": 4}', name='get_n_day_weather_forecast'), type='function'),
 ChatCompletionMessageFunctionToolCall(id='call_YAnH0VRB3oqjqivcGj3Cd8YA', function=Function(arguments='{"location": "Glasgow, UK", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function')]

## モデルが生成した引数で関数を呼び出す方法

次の例では、入力がモデルによって生成された関数を実行する方法を実演し、これを使用してデータベースに関する質問に答えることができるエージェントを実装します。簡単にするために、[Chinook サンプルデータベース](https://www.sqlitetutorial.net/sqlite-sample-database/)を使用します。

*注意:* モデルは完全に信頼できる正しいSQLを生成するわけではないため、本番環境でのSQL生成は高リスクとなる可能性があります。

### SQL クエリを実行する関数の指定

まず、SQLite データベースからデータを抽出するための便利なユーティリティ関数をいくつか定義しましょう。

In [14]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

Opened database successfully


In [15]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


これで、これらのユーティリティ関数を使用してデータベーススキーマの表現を抽出できるようになりました。

In [16]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

前回と同様に、APIに引数を生成してもらいたい関数の仕様を定義します。関数仕様にデータベーススキーマを挿入していることに注目してください。これは、モデルが理解するために重要になります。

In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

### SQLクエリの実行

それでは、実際にデータベースに対してクエリを実行する関数を実装しましょう。

In [18]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

##### Chat Completions APIを使用して関数呼び出しを実行する手順：

**ステップ1**: モデルがツールを使用することを選択する可能性のあるコンテンツでモデルにプロンプトを送信します。関数名やシグネチャなどのツールの説明は'Tools'リストで定義され、API呼び出しでモデルに渡されます。選択された場合、関数名とパラメータがレスポンスに含まれます。<br>
  
**ステップ2**: モデルが関数を呼び出したいかどうかをプログラム的にチェックします。trueの場合、ステップ3に進みます。<br>  
**ステップ3**: レスポンスから関数名とパラメータを抽出し、パラメータを使って関数を呼び出します。結果をメッセージに追加します。<br>    
**ステップ4**: メッセージリストを使ってchat completions APIを呼び出し、レスポンスを取得します。

In [21]:
# Step #1: Prompt with content that may result in function call. In this case the model can identify the information requested by the user is potentially available in the database schema passed to the model in Tools description. 
messages = [{
    "role":"user", 
    "content": "What is the name of the album with the most tracks?"
}]

response = client.chat.completions.create(
    model=GPT_MODEL, 
    messages=messages, 
    tools=tools, 
    tool_choice="auto"
)

# Append the message to messages list
response_message = response.choices[0].message 
messages.append(response_message.to_dict())
pretty_print_conversation(messages)

user: What is the name of the album with the most tracks?

assistant: [{'id': 'call_pGRtZZGfd2o41GHlZcEdB9he', 'function': {'arguments': '{"query":"WITH track_counts AS (\\n  SELECT a.AlbumId, a.Title, COUNT(t.TrackId) AS track_count\\n  FROM Album a\\n  JOIN Track t ON t.AlbumId = a.AlbumId\\n  GROUP BY a.AlbumId, a.Title\\n)\\nSELECT Title\\nFROM track_counts\\nWHERE track_count = (SELECT MAX(track_count) FROM track_counts);"}', 'name': 'ask_database'}, 'type': 'function'}]



In [22]:
# Step 2: determine if the response from the model includes a tool call.   
tool_calls = response_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = json.loads(tool_calls[0].function.arguments)['query']

    # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_query_string)
        
        messages.append({
            "role":"tool", 
            "tool_call_id":tool_call_id, 
            "name": tool_function_name, 
            "content":results
        })
        
        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(f"Result found in database: {model_response_with_function_call.choices[0].message.content}")
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    print(response_message.content) 

Result found in database: Greatest Hits


## 次のステップ

ナレッジベースと対話的にやり取りするために、Chat Completions APIと関数を使用してナレッジ検索を行う方法を実演する、他の[ノートブック](How_to_call_functions_for_knowledge_retrieval.ipynb)をご覧ください。